# Blockchain analysis

In case you know nothing about the Bitcoin and Blockchain, you can start by watching the following video.

In [4]:
import IPython.display
IPython.display.HTML('<iframe width="750" height="430" src="https://www.youtube.com/embed/Lx9zgZCMqXE?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

## Basic setup

Here we will import the `pyspark` module and set up a `SparkSession`. By default, we'll use a `SparkSession` running locally, with one Spark executor; we're dealing with small data, so it doesn't make sense to run against a cluster, but the `local[1]` can be changed with the ip of the Spark cluster.


In [2]:
sc = spark.sparkContext
sc._conf.getAll()

[(u'spark.driver.host', u'10.1.4.234'),
 (u'spark.ui.reverseProxyUrl', u'/'),
 (u'spark.app.id', u'app-20170921114512-0001'),
 (u'spark.master', u'spark://10.1.2.102:7077'),
 (u'spark.executor.id', u'driver'),
 (u'spark.app.name', u'PySparkShell'),
 (u'spark.driver.port', u'38073'),
 (u'spark.jars',
  u'file:/opt/spark/.ivy2/jars/graphframes_graphframes-0.5.0-spark2.1-s_2.11.jar,file:/opt/spark/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/opt/spark/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/opt/spark/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/opt/spark/.ivy2/jars/org.slf4j_slf4j-api-1.7.7.jar'),
 (u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.pyFiles',
  u'/opt/spark/.ivy2/jars/graphframes_graphframes-0.5.0-spark2.1-s_2.11.jar,/opt/spark/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.j

In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import regexp_replace

# spark = SparkSession.builder \
#                     .master("local[4]") \
#                     .config("spark.driver.memory", "4g") \
#                     .getOrCreate()

# sc = spark.sparkContext

## Loading the data

To obtain the graph representing the transaction in the Bitcoin network, we need to load set of nodes representing the wallets (fingerprints of the public keys) and the set of edges representing each transaction. For this example we will use two parquet files that were generated from the blockchain data by this [convertor](https://github.com/Jiri-Kremser/bitcoin-insights/tree/master/parquet-converter).

In [2]:
raw_nodes = spark.read.load("/tmp/data/nodes.parquet") \
                      .withColumnRenamed("_1", "id") \
                      .withColumnRenamed("_2", "Wallet")
raw_nodes.show(5)

+---+--------------------+
| id|              Wallet|
+---+--------------------+
|  0|bitcoinaddress_93...|
|  1|bitcoinaddress_4D...|
|  2|bitcoinaddress_BE...|
|  3|bitcoinaddress_4B...|
|  4|bitcoinaddress_44...|
+---+--------------------+
only showing top 5 rows



As you can see, each record in the wallet column contains a string `bitcoinaddress_<hash>`, where the hash is the actual address of the wallet. Let's remove the redundant prefix.

In [3]:
nodes = raw_nodes.withColumn("Wallet", regexp_replace("Wallet", "bitcoinaddress_", "")).cache()
nodes.show(5)

+---+--------------------+
| id|              Wallet|
+---+--------------------+
|  0|9303DBB4C75A56057...|
|  1|4D3826A813A4B4E9B...|
|  2|BECC6154EEF33464E...|
|  3|4B5E0300F11C2932F...|
|  4|44730B80C9D5EF65D...|
+---+--------------------+
only showing top 5 rows



As you can see, each record in the wallet column contains a string `bitcoinaddress_<hash>`, where the hash is the actual address of the wallet. Let's remove the redundant prefix.

We can also verify, that these addresses are real on https://blockchain.info/address/. 

Example:
 1. get a random wallet address
 1. create the link from the address

In [9]:
random_address = nodes.rdd.takeSample(False, 1)[0][1]
IPython.display.Markdown('link of the random wallet: https://blockchain.info/address/' + random_address)

link of the random wallet: https://blockchain.info/address/B4F37D584299E5B1B1C4CF24C463DBBF524DDC2D

In [4]:
raw_edges = spark.read.load("/tmp/data/edges.parquet") \
                      .withColumnRenamed("srcId", "src") \
                      .withColumnRenamed("dstId", "dst") \
                      .cache()
raw_edges.show(5)
raw_edges.count()

+------+------+-----+
|   src|   dst| attr|
+------+------+-----+
|150102|107378|input|
|470403|107378|input|
|232249| 97703|input|
|539070| 97703|input|
|131174|176711|input|
+------+------+-----+
only showing top 5 rows



2087249

## Data cleansing

Remove the self-loops.

In [5]:
edges = raw_edges.filter("src != dst")
edges.show(5)
edges.count()

+------+------+-----+
|   src|   dst| attr|
+------+------+-----+
|150102|107378|input|
|470403|107378|input|
|232249| 97703|input|
|539070| 97703|input|
|131174|176711|input|
+------+------+-----+
only showing top 5 rows



2081796

## Constructing the graph representation

Spark contains API for graph processing. It's called [graphx](https://spark.apache.org/graphx/) and it also comes with multiple built-in algorithms like page-rank. It uses the [Pregel API](https://spark.apache.org/docs/latest/graphx-programming-guide.html#pregel-api).

In [6]:
from graphframes import *

g = GraphFrame(nodes, edges).cache()

#### Get the top 10 wallets with respect to the transaction count

First, by sorting the nodes by `inDegree` which corresponds to the number of transactions received.

In [17]:
vertexInDegrees = g.inDegrees
vertexInDegrees.join(nodes, vertexInDegrees.id == nodes.id) \
               .drop("id") \
               .orderBy("inDegree", ascending=False) \
               .take(10)

[Row(inDegree=121885, Wallet=u'F4B004C3CA2E7F96F9FC5BCA767708967AF67A44'),
 Row(inDegree=69445, Wallet=u'E1BB16A26D591FD766C1B23FAEC067301AFA8A07'),
 Row(inDegree=68626, Wallet=u'5605C6DC8A9672A014225BAC565DB25BCEC649A2'),
 Row(inDegree=5744, Wallet=u'66A731E9FEB460A3A48B5C56BB635B3A409DBD56'),
 Row(inDegree=3100, Wallet=u'17EBC6064CB035D84DC177CE763D2C93FD5556C6'),
 Row(inDegree=2338, Wallet=u'A84458A0E8F009B3780CAC779873D298CF22BE8A'),
 Row(inDegree=2113, Wallet=u'EB9A27CF65B32AABA6FD08D7C4DE65F4C7CF9361'),
 Row(inDegree=2080, Wallet=u'EFB0CD7206CCF14275643097970DB8A38497B61D'),
 Row(inDegree=2002, Wallet=u'9B3A7C3A61712270055C1E4AC6FF2704D3ACB1E0'),
 Row(inDegree=1816, Wallet=u'E1E62CCCDF52C20DA0958A68F1D90A381792D669')]

Then by using the `outDegree` ~ # transactions sent

In [20]:
vertexOutDegrees = g.outDegrees
senders = vertexOutDegrees.join(nodes, vertexOutDegrees.id == nodes.id) \
                          .drop("id") \
                          .orderBy("outDegree", ascending=False)
senders.take(10)

[Row(outDegree=1994, Wallet=u'7B1DD94268DF8E11BA27AB1C99C61E914C717246'),
 Row(outDegree=1965, Wallet=u'CBD3148D93C205F862599C080556A2A531146A3B'),
 Row(outDegree=1952, Wallet=u'9B3A7C3A61712270055C1E4AC6FF2704D3ACB1E0'),
 Row(outDegree=1694, Wallet=u'580740AC5A1B84C24D7EDB3F3AFC635BDFFB587B'),
 Row(outDegree=1675, Wallet=u'826AC9812C2BE5FE349A61ACD393F1F38E8C7D2D'),
 Row(outDegree=1590, Wallet=u'17EBC6064CB035D84DC177CE763D2C93FD5556C6'),
 Row(outDegree=1577, Wallet=u'EFB0CD7206CCF14275643097970DB8A38497B61D'),
 Row(outDegree=1568, Wallet=u'A84458A0E8F009B3780CAC779873D298CF22BE8A'),
 Row(outDegree=1559, Wallet=u'64B7D877B3DB608FD62EC8034770FD0AEFE9975A'),
 Row(outDegree=1559, Wallet=u'5445DAC9A29FCDE8C7C896CF063923FBED4F5D2C')]

You can verify on blockchain.info that the actual number of transaction is lower than what we have just calculated. This stems from the fact how Bitcoin works, when sending BTC from a wallet to another one, it actually sends all the BTC and the receiving node will sends back the rest. We are oversimplyfying here and you can find the details [here](https://en.bitcoin.it/wiki/Transaction) or [here](https://bitcoin.stackexchange.com/questions/9007/why-are-there-two-transaction-outputs-when-sending-to-one-address).

#### Find circles of length 2

In [25]:
# motifs = g.find("(a)-[]->(b); (b)-[]->(a)")
# motifs.count()
# # motifs.show(5)

motifs = g.find("(a)-[]->(b)") \
          .filter("a = b")
motifs.count()
# motifs.show(5)

0

#### Resource consuming foo

In [18]:
# this fails with OOM error
#results = g.pageRank(resetProbability=0.15, maxIter=1)
#results.vertices.select("id", "pagerank").show()
#results.edges.select("src", "dst", "weight").show()

# g.labelPropagation(maxIter=1)

# this would be nice display(ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(20))

Py4JJavaError: An error occurred while calling o322.run.
: org.apache.spark.SparkException: Job 17 cancelled because killed via the Web UI
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1439)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply$mcVI$sp(DAGScheduler.scala:1428)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1421)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleStageCancellation$1.apply(DAGScheduler.scala:1421)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofInt.foreach(ArrayOps.scala:234)
	at org.apache.spark.scheduler.DAGScheduler.handleStageCancellation(DAGScheduler.scala:1421)
	at org.apache.spark.scheduler.DAGSchedulerE

## Visualization of a sub-graph

Our data contain a lot of transactions (2 087 249 transactions among 546 651 wallets) so let's show only a small fraction of the transaction graph. We will show all the outgoing transaction of particular bitcoin address.

In [30]:
from pyspark.sql.functions import col
import random

# feel free to use any address that is present in the dataset
address = senders.take(2000)[1999].Wallet

sub_graph = g.find("(src)-[e]->(dst)") \
             .filter(col('src.Wallet') == address)
    
def node_to_dict(r):
    return {
        'id': r[0],
        'label': r[1],
        'x': random.uniform(0,1),
        'y': random.uniform(0,1),
        'size': random.uniform(0.2,1)
    }

sub_nodes = sub_graph.select("dst.id", "dst.Wallet").distinct()
sub_edges = sub_graph.select("e.src", "e.dst")

target_nodes_dict = map(node_to_dict, sub_nodes.collect())

def edge_to_dict(i, r):
    return {
        'id': i,
        'source': r[0],
        'target': r[1]
    }

sub_edges_dict = [edge_to_dict(i, r) for i, r in enumerate(sub_edges.collect())]

target_nodes_dict.append({
    'id': sub_edges.first()['src'],
    'label': address,
    'color': '#999',
    'x': -1,
    'y': 0.5,
    'size': 2
})

Now we are ready to show the data using the [sigmajs](sigmajs.org) library.

In [22]:
%%javascript
require.config({
    paths: {
        sigmajs: 'https://cdnjs.cloudflare.com/ajax/libs/sigma.js/1.2.0/sigma.min'
    }
});

require(['sigmajs']);

<IPython.core.display.Javascript object>

In [31]:
from IPython.core.display import display, HTML
from string import Template
import json

js_text_template = Template(open('js/sigma-graph.js','r').read())

graph_data = { 'nodes': target_nodes_dict, 'edges': sub_edges_dict }

js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div'})

html_template = Template('''
<div id="graph-div" style="height:400px"></div>
<script> $js_text </script>
''')

HTML(html_template.substitute({'js_text': js_text}))

In [52]:
sub_g = GraphFrame(sub_nodes.union(sub_graph.select("src.id", "src.Wallet").distinct()), sub_edges).cache()
sub_g

# #results = g.pageRank(resetProbability=0.15, maxIter=1)
# #results.vertices.select("id", "pagerank").show()
# #results.edges.select("src", "dst", "weight").show()
# # g.labelPropagation(maxIter)

# # this would be nice display(ranks.vertices.orderBy(ranks.vertices.pagerank.desc()).limit(20))


labeled = sub_g.triangleCount()
labeled.show()

# results = sub_g.pageRank(resetProbability=0.15, maxIter=3)

+-----+------+--------------------+
|count|    id|              Wallet|
+-----+------+--------------------+
|    0| 98630|6C8EB8EB53EA37059...|
|    0|299908|E3F08554C4D797AB1...|
|    0| 85806|E59E4A4C94E54B76C...|
|    0|291988|4863FCA534DF6069E...|
|    0| 26673|99C37F14F5CD2AA5B...|
|    0|228133|AB4E11D8B737B9F29...|
|    0|189874|1F9DA1BFF29639909...|
|    0| 90723|865BA68EB97FA9623...|
|    0| 36789|74983C5871243018F...|
|    0|195799|EC8B7C31883F9E134...|
|    0|531266|3F621757C623E1927...|
|    0| 49633|4DFD4A60C3D5F6429...|
|    0|402628|AA45834F735090FED...|
|    0|241675|28216E7DA03214035...|
|    0|152448|0EBFCF01A516F0962...|
|    0|451006|28A18D3B87BA226BF...|
|    0|192506|C1DA21F74CD0F721D...|
|    0|143918|A4B1FDB4537D779C7...|
|    0| 73489|52DC8BD47695B6D54...|
|    0|449200|F4B004C3CA2E7F96F...|
+-----+------+--------------------+
only showing top 20 rows



In [7]:
triangles = g.triangleCount()
triangles.show()

Py4JJavaError: An error occurred while calling o57.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 19.1 failed 4 times, most recent failure: Lost task 4.3 in stage 19.1 (TID 2023, 10.1.3.190, executor 3): java.io.FileNotFoundException: /tmp/spark-afcf462d-4e08-4cfc-8b5f-19d6bd2e2a30/executor-c44db3ca-9502-4710-b4b9-5de572c135ab/blockmgr-6b74d2d0-9b70-471b-9c92-9233c81c662e/05/shuffle_7_4_0.index.3954f15e-8dda-4e89-a3b4-a81a7249fa48 (No such file or directory)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(FileOutputStream.java:270)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:213)
	at java.io.FileOutputStream.<init>(FileOutputStream.java:162)
	at org.apache.spark.shuffle.IndexShuffleBlockResolver.writeIndexFileAndCommit(IndexShuffleBlockResolver.scala:144)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:127)
	at org.apache.spark.sched

In [49]:
from graphframes import examples
# f = Graphs(sqlContext).friends()

ImportError: No module named lib

In [25]:
labeled

# results.vertices.orderBy(results.vertices.pagerank.asc()).limit(20).show()
# results.edges.select("src", "dst", "weight").show()

NameError: name 'labeled' is not defined

In [ ]:
labeled = g.labelPropagation(maxIter=3)
g.show()

In [ ]:
g.show()